# CPRD Notebook:
## Pre-training SurvivEHR-CR model 


In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
from tqdm import tqdm
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.run_experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import os
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

 # TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

Using device: cuda.
env: SLURM_NTASKS_PER_NODE=28


## Choosing configurations
The default configuration is for pre-training. Here we modify as necesssary

Here we choose to load in the configuration for a small **pre-trained** 11.4M parameter model, named "CR_11M". We specfiy the `zero-shot` experiment type, which will lead to running the ```SupervisedExperiment```. 

We tell this experiment that no further training is needed. Additionally, we do choose to perform testing (true by default). As this is a supervised model, this tests the ability to predict the outcomes of interest. In this notebook, this is chosen to be those of the cohort study for predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population, and we add the folder containing this dataset to the configuration. 

```Note: As this is a supervised dataset, we need to tell the DataModule that the last event observed is a target and must be stripped. This is done by passing a list of targets to the configuration, overriding the null default. This lets the DataModule know that it should process batches as supervised.```

We set the number of workers to be appropriate for the number of CPUs available to reduce bottlenecking, and tell the experiment that we do not want to limit the number of testing batches. In addition, we specify where we want any checkpoints to be saved to avoid bloating the repository.

# Run small (11M) Competing-Risk model experiment

```

```

In [3]:
pre_trained_model_ids = ['SurvivEHR-cr-small', 'SurvivEHR-cr-small-v1', 'SurvivEHR-cr', 'SurvivEHR-cr-v1', 'SurvivEHR-cr-v1-v1', 'SurvivEHR-cr-384', 'SurvivEHR-cr-384-v1', 'crPreTrain_small_1337',
                        'SurvivEHR-cr-small-192', "SurvivEHR-cr-small-192-v1"]

pre_trained_model = pre_trained_model_ids[-1]
print(pre_trained_model)

SurvivEHR-cr-small-192-v1


In [4]:
wandb.finish()

# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../../confs", job_name="testing_notebook"):
    cfg = compose(config_name="config_CompetingRisk11M", 
                  overrides=[# Experiment setup
                             f"experiment.run_id='{pre_trained_model}'",
                             "experiment.train=False",
                             "experiment.test=True",
                             # Dataloader
                             "data.batch_size=128",
                             "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                             "data.min_workers=3",
                             # "data.global_diagnoses=True",
                             # Optimiser
                             "optim.limit_test_batches=0.035",
                             # Model
                             "transformer.n_embd=192",  #384
                             # "transformer.block_size=512", 
                            ]
                 )     

# print(OmegaConf.to_yaml(cfg))
model, dm = run(cfg)
print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")

wandb.finish()

INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/. This will be loaded in causal form.
INFO:root:====================================================================================================
INFO:root:Creating unsupervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_test.pickle
INF

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        Test:Cinter        │    0.9787275421057963     │
│       Test:Cinter+0       │    0.9535032577335341     │
│       Test:Cinter+1       │    0.8958194011884513     │
│      Test:Cinter+10       │    0.7646560061472428     │
│      Test:Cinter+13       │    0.7330274976580149     │
│      Test:Cinter+16       │    0.6945452986077362     │
│      Test:Cinter+19       │    0.7070922089933495     │
│       Test:Cinter+2       │    0.8744953913221967     │
│       Test:Cinter+3       │    0.8559257247470172     │
│       Test:Cinter+4       │    0.8257447465678294     │
│       Test:Cinter+7       │    0.7530207013096748     │
│      Test:Cintra100       │    0.7940430925221799     │
│      Test:Cintra101       │    0.9353612167300381     │
│      Test:Cintra104       │    0.8365019011406845     │
│      Test:Cintra106       │   0.008871989860583017    │
│      Test:Cintra107       │    0.8431558935361215     │
│       Test:Cintra11       │    0.40304182509505704    │
│      Test:Cintra110       │    0.8056083650190113     │
│      Test:Cintra112       │    0.9358571664737972     │
│      Test:Cintra113       │    0.9904942965779467     │
│      Test:Cintra114       │    0.8726235741444867     │
│      Test:Cintra115       │     0.985044359949303     │
│      Test:Cintra116       │    0.9890494296577947     │
│      Test:Cintra118       │    0.9787949692892658     │
│      Test:Cintra119       │    0.9730671736375159     │
│      Test:Cintra120       │    0.8814955640050698     │
│      Test:Cintra121       │    0.9892268694550063     │
│      Test:Cintra123       │    0.9644683361741184     │
│      Test:Cintra124       │    0.9581749049429658     │
│      Test:Cintra126       │    0.9121854064819844     │
│      Test:Cintra127       │    0.9795627376425855     │
│      Test:Cintra129       │    0.9019672673169118     │
│      Test:Cintra130       │    0.9819391634980988     │
│      Test:Cintra131       │    0.9840889148873941     │
│      Test:Cintra133       │    0.9413362303096146     │
│      Test:Cintra134       │    0.9365710335292085     │
│      Test:Cintra135       │    0.9640895648500211     │
│      Test:Cintra136       │    0.9576530231864607     │
│      Test:Cintra141       │    0.6882129277566541     │
│      Test:Cintra142       │    0.9817663325267888     │
│      Test:Cintra143       │     0.973729692360871     │
│      Test:Cintra144       │    0.9874049429657795     │
│      Test:Cintra145       │    0.9898605830164766     │
│      Test:Cintra146       │    0.9367088607594934     │
│      Test:Cintra148       │    0.9655827979546349     │
│      Test:Cintra149       │    0.9891935161096657     │
│      Test:Cintra150       │    0.9853992395437265     │
│      Test:Cintra151       │    0.9979059899707942     │
│      Test:Cintra152       │    0.9574436969874233     │
│      Test:Cintra153       │    0.9971718568331085     │
│      Test:Cintra154       │    0.9488973384030416     │
│      Test:Cintra155       │    0.9792232482346553     │
│      Test:Cintra156       │    0.9925072690673229     │
│      Test:Cintra157       │    0.9616535878974195     │
│      Test:Cintra158       │    0.9983598001938419     │
│      Test:Cintra159       │    0.9923954372623573     │
│      Test:Cintra160       │     0.980264349085642     │
│      Test:Cintra161       │    0.9958520566885586     │
│      Test:Cintra162       │    0.9898605830164765     │
│      Test:Cintra163       │    0.9826232187910884     │
│      Test:Cintra164       │     0.996397838703222     │
│      Test:Cintra165       │    0.9913229989275621     │
│      Test:Cintra166       │    0.9945681694731123     │
│      Test:Cintra167       │    0.9797734310689509     │
│      Test:Cintra168       │    0.9791128010139425     │
│      Test:Cintra169       │     0.986850443599493     

Loaded model with 2.891654 M parameters


Test:Cinter,▁
Test:Cinter+0,▁
Test:Cinter+1,▁
Test:Cinter+10,▁
Test:Cinter+13,▁
Test:Cinter+16,▁
Test:Cinter+19,▁
Test:Cinter+2,▁
Test:Cinter+3,▁
Test:Cinter+4,▁
Test:Cinter+7,▁


In [7]:
wandb.finish()
print(pre_trained_model)

SurvivEHR-cr-small-192


In [8]:
display(dm.encode(['IHDINCLUDINGMI_OPTIMALV2', 'ISCHAEMICSTROKE_V2', 'MINFARCTION', 'STROKEUNSPECIFIED_V2', 'STROKE_HAEMRGIC']))
display(dm.encode(['HYPERTENSION']))
# display(dm.decode([95, 175, 263,249]).split(" "))

[95, 41, 67, 65, 28]

[129]

In [9]:
dm.tokenizer._event_counts["EVENT"][-5:].to_list()

['NSAIDS_oral_OPTIMAL_final',
 'Diastolic_blood_pressure_5',
 'Systolic_blood_pressure_4',
 'Statins',
 'Lipid_lowering_drugs_Optimal']

In [10]:
raise NotImplementedError

NotImplementedError: 

In [ ]:
# all_nondiagnosis = [cn for cn in dm.tokenizer._event_counts["EVENT"] if cn.upper() != cn]
# display([cn for cn in all_nondiagnosis if "Warf" in cn])

# Get the data from the callback

In [ ]:
api = wandb.Api()

In [ ]:
# run = api.run("SurvivEHR/test_causal_eval")

In [5]:
raw_data_from_wandb = {
  "Test:Cinter": 0.9787275421057964,
  "Test:Cinter+0": 0.953503257733534,
  "Test:Cinter+1": 0.8958194011884513,
  "Test:Cinter+10": 0.7646560061472428,
  "Test:Cinter+13": 0.7330274976580149,
  "Test:Cinter+16": 0.6945452986077362,
  "Test:Cinter+19": 0.7070922089933495,
  "Test:Cinter+2": 0.8744953913221967,
  "Test:Cinter+3": 0.8559257247470172,
  "Test:Cinter+4": 0.8257447465678294,
  "Test:Cinter+7": 0.7530207013096748,
  "Test:Cintra100": 0.7940430925221799,
  "Test:Cintra101": 0.935361216730038,
  "Test:Cintra104": 0.8365019011406845,
  "Test:Cintra106": 0.008871989860583017,
  "Test:Cintra107": 0.8431558935361215,
  "Test:Cintra11": 0.40304182509505704,
  "Test:Cintra110": 0.8056083650190113,
  "Test:Cintra112": 0.9358571664737972,
  "Test:Cintra113": 0.9904942965779469,
  "Test:Cintra114": 0.8726235741444867,
  "Test:Cintra115": 0.985044359949303,
  "Test:Cintra116": 0.9890494296577949,
  "Test:Cintra118": 0.9787949692892658,
  "Test:Cintra119": 0.973067173637516,
  "Test:Cintra120": 0.8814955640050698,
  "Test:Cintra121": 0.9892268694550064,
  "Test:Cintra123": 0.9644683361741184,
  "Test:Cintra124": 0.9581749049429658,
  "Test:Cintra126": 0.9121854064819844,
  "Test:Cintra127": 0.9795627376425856,
  "Test:Cintra129": 0.9019672673169118,
  "Test:Cintra130": 0.9819391634980988,
  "Test:Cintra131": 0.984088914887394,
  "Test:Cintra133": 0.9413362303096146,
  "Test:Cintra134": 0.9365710335292085,
  "Test:Cintra135": 0.9640895648500212,
  "Test:Cintra136": 0.9576530231864608,
  "Test:Cintra141": 0.6882129277566541,
  "Test:Cintra142": 0.9817663325267888,
  "Test:Cintra143": 0.973729692360871,
  "Test:Cintra144": 0.9874049429657796,
  "Test:Cintra145": 0.9898605830164766,
  "Test:Cintra146": 0.9367088607594934,
  "Test:Cintra148": 0.9655827979546348,
  "Test:Cintra149": 0.9891935161096655,
  "Test:Cintra150": 0.9853992395437264,
  "Test:Cintra151": 0.9979059899707942,
  "Test:Cintra152": 0.9574436969874232,
  "Test:Cintra153": 0.9971718568331084,
  "Test:Cintra154": 0.9488973384030416,
  "Test:Cintra155": 0.9792232482346552,
  "Test:Cintra156": 0.9925072690673228,
  "Test:Cintra157": 0.9616535878974196,
  "Test:Cintra158": 0.998359800193842,
  "Test:Cintra159": 0.9923954372623572,
  "Test:Cintra160": 0.980264349085642,
  "Test:Cintra161": 0.9958520566885586,
  "Test:Cintra162": 0.9898605830164764,
  "Test:Cintra163": 0.9826232187910884,
  "Test:Cintra164": 0.996397838703222,
  "Test:Cintra165": 0.991322998927562,
  "Test:Cintra166": 0.9945681694731124,
  "Test:Cintra167": 0.9797734310689508,
  "Test:Cintra168": 0.9791128010139424,
  "Test:Cintra169": 0.986850443599493,
  "Test:Cintra170": 0.9959175505303184,
  "Test:Cintra171": 0.976056388550112,
  "Test:Cintra172": 0.963950522212062,
  "Test:Cintra173": 0.997229345359154,
  "Test:Cintra174": 0.9994568169473113,
  "Test:Cintra175": 0.9840515420363328,
  "Test:Cintra176": 0.9722574285311938,
  "Test:Cintra177": 0.9904467680608364,
  "Test:Cintra178": 0.9964592512121032,
  "Test:Cintra179": 0.986183259251326,
  "Test:Cintra180": 0.9961977186311792,
  "Test:Cintra181": 0.9920599418474616,
  "Test:Cintra182": 0.9853929024081116,
  "Test:Cintra183": 0.9978312913674136,
  "Test:Cintra184": 0.9763751584283904,
  "Test:Cintra185": 0.9224334600760454,
  "Test:Cintra186": 0.998562917277926,
  "Test:Cintra187": 0.999426070736782,
  "Test:Cintra188": 0.993029150823828,
  "Test:Cintra189": 0.9791048941291384,
  "Test:Cintra190": 0.995487959442332,
  "Test:Cintra191": 0.9776932826362484,
  "Test:Cintra192": 0.980988593155894,
  "Test:Cintra193": 0.9848995111352536,
  "Test:Cintra194": 0.99520581914366,
  "Test:Cintra195": 0.9805323193916352,
  "Test:Cintra196": 0.9914172569932002,
  "Test:Cintra197": 0.9555893536121672,
  "Test:Cintra198": 0.9464864103647376,
  "Test:Cintra199": 0.981793782151644,
  "Test:Cintra200": 0.9897715488797412,
  "Test:Cintra201": 0.99574403733149,
  "Test:Cintra202": 0.9874973018745752,
  "Test:Cintra203": 0.9959864807773556,
  "Test:Cintra204": 0.9994705684169998,
  "Test:Cintra205": 0.961402422849058,
  "Test:Cintra206": 0.9934133708571606,
  "Test:Cintra207": 0.9841659625404856,
  "Test:Cintra208": 0.999393632427111,
  "Test:Cintra209": 0.9917317254257878,
  "Test:Cintra210": 0.9860933134466318,
  "Test:Cintra211": 0.9950168859079416,
  "Test:Cintra212": 0.9981661022336872,
  "Test:Cintra213": 0.993587197094376,
  "Test:Cintra214": 0.9938393415796304,
  "Test:Cintra215": 0.996142613104093,
  "Test:Cintra216": 0.987068282636249,
  "Test:Cintra217": 0.9912601589389614,
  "Test:Cintra218": 0.9912020204823628,
  "Test:Cintra219": 0.994372048559946,
  "Test:Cintra220": 0.9998428190772723,
  "Test:Cintra221": 0.989971482889734,
  "Test:Cintra222": 0.990964578747248,
  "Test:Cintra223": 0.9869378523665888,
  "Test:Cintra224": 0.9974261479964897,
  "Test:Cintra225": 0.999833233273297,
  "Test:Cintra226": 0.9989237344059312,
  "Test:Cintra227": 0.9989831107967102,
  "Test:Cintra228": 0.985605649103748,
  "Test:Cintra229": 0.9945278507310544,
  "Test:Cintra230": 0.9948546242173124,
  "Test:Cintra231": 0.01084857263564486,
  "Test:Cintra232": 0.9996519945865812,
  "Test:Cintra233": 0.9962638452636802,
  "Test:Cintra234": 0.9889718691772788,
  "Test:Cintra235": 0.993107948832346,
  "Test:Cintra236": 0.999371684339394,
  "Test:Cintra237": 0.9999805343957228,
  "Test:Cintra238": 0.999812790900346,
  "Test:Cintra239": 0.9996017880998868,
  "Test:Cintra240": 0.9942125668199608,
  "Test:Cintra241": 0.994427281368818,
  "Test:Cintra242": 0.9913078851677782,
  "Test:Cintra243": 0.9995133079847908,
  "Test:Cintra244": 0.9922433460076032,
  "Test:Cintra245": 0.998105848803398,
  "Test:Cintra246": 0.9995058406885284,
  "Test:Cintra247": 0.9898189793125474,
  "Test:Cintra248": 0.997593759207552,
  "Test:Cintra249": 0.99740070030719,
  "Test:Cintra25": 0.9163498098859316,
  "Test:Cintra250": 0.9922580054056528,
  "Test:Cintra251": 0.9929457238423198,
  "Test:Cintra252": 0.997756311444524,
  "Test:Cintra253": 0.9986125008338336,
  "Test:Cintra254": 0.995801472014028,
  "Test:Cintra255": 0.9853883758826716,
  "Test:Cintra256": 0.9914071791092272,
  "Test:Cintra257": 0.9961977186311788,
  "Test:Cintra258": 1,
  "Test:Cintra259": 0.9965627376425826,
  "Test:Cintra260": 0.9934657452569176,
  "Test:Cintra261": 0.9964564305593652,
  "Test:Cintra262": 0.9986013127715232,
  "Test:Cintra263": 0.9997470499194968,
  "Test:Cintra264": 0.9970169093325892,
  "Test:Cintra27": 0.8593155893536122,
  "Test:Cintra31": 0.8821292775665399,
  "Test:Cintra34": 0.4917617237008873,
  "Test:Cintra38": 0.49429657794676807,
  "Test:Cintra39": 0.9239543726235742,
  "Test:Cintra40": 0.7173637515842839,
  "Test:Cintra41": 0.5190114068441065,
  "Test:Cintra44": 0.7984790874524715,
  "Test:Cintra47": 0.8136882129277566,
  "Test:Cintra48": 0.6787072243346007,
  "Test:Cintra51": 0.785551330798479,
  "Test:Cintra55": 0.9809885931558936,
  "Test:Cintra57": 0.005703422053231939,
  "Test:Cintra58": 0.873574144486692,
  "Test:Cintra60": 0.6197718631178707,
  "Test:Cintra63": 0.9771863117870724,
  "Test:Cintra65": 0.7870722433460076,
  "Test:Cintra67": 0.6349809885931559,
  "Test:Cintra70": 0.8326996197718631,
  "Test:Cintra71": 0.311787072243346,
  "Test:Cintra75": 0.7727080692860162,
  "Test:Cintra78": 0.7585551330798479,
  "Test:Cintra79": 0.8041825095057034,
  "Test:Cintra80": 0.997827267789245,
  "Test:Cintra81": 0.752851711026616,
  "Test:Cintra82": 0.8250950570342206,
  "Test:Cintra83": 0.7756653992395437,
  "Test:Cintra84": 0.8657794676806084,
  "Test:Cintra85": 0.7022813688212928,
  "Test:Cintra89": 0.7699619771863119,
  "Test:Cintra91": 0.8504435994930292,
  "Test:Cintra93": 0.7591888466413182,
  "Test:Cintra94": 0.8359015409245546,
  "Test:Cintra95": 1,
  "Test:Cintra97": 0.8408473655621945,
  "Test:Cintra98": 0.9301330798479088,
  "Test:Cintra99": 0.9847908745247148,
  "Test:base_Cinter": 0.8626828301681702,
  "Test:base_Cinter+0": 0.8549209927966827,
  "Test:base_Cinter+1": 0.8549209927966827,
  "Test:base_Cinter+10": 0.8498032617479898,
  "Test:base_Cinter+13": 0.848335813082052,
  "Test:base_Cinter+16": 0.8504674233111297,
  "Test:base_Cinter+19": 0.8518284716003348,
  "Test:base_Cinter+2": 0.8554057884602225,
  "Test:base_Cinter+3": 0.8562878467821434,
  "Test:base_Cinter+4": 0.8553855002076877,
  "Test:base_Cinter+7": 0.8501297603959204,
  "Test:base_Cintra100": 0.376425855513308,
  "Test:base_Cintra101": 0.38022813688212925,
  "Test:base_Cintra104": 0.3916349809885931,
  "Test:base_Cintra106": 0.3992395437262359,
  "Test:base_Cintra107": 0.4030418250950571,
  "Test:base_Cintra11": 0.03802281368821293,
  "Test:base_Cintra110": 0.41444866920152096,
  "Test:base_Cintra112": 0.4220532319391635,
  "Test:base_Cintra113": 0.4258555133079848,
  "Test:base_Cintra114": 0.4296577946768061,
  "Test:base_Cintra115": 0.4334600760456273,
  "Test:base_Cintra116": 0.4372623574144487,
  "Test:base_Cintra118": 0.4448669201520912,
  "Test:base_Cintra119": 0.44866920152091183,
  "Test:base_Cintra120": 0.45247148288973377,
  "Test:base_Cintra121": 0.4562737642585551,
  "Test:base_Cintra123": 0.4638783269961979,
  "Test:base_Cintra124": 0.467680608365019,
  "Test:base_Cintra126": 0.4752851711026616,
  "Test:base_Cintra127": 0.479087452471483,
  "Test:base_Cintra129": 0.48669201520912553,
  "Test:base_Cintra130": 0.49049429657794685,
  "Test:base_Cintra131": 0.49429657794676896,
  "Test:base_Cintra133": 0.5019011406844106,
  "Test:base_Cintra134": 0.5057034220532319,
  "Test:base_Cintra135": 0.5095057034220533,
  "Test:base_Cintra136": 0.5133079847908741,
  "Test:base_Cintra141": 0.532319391634981,
  "Test:base_Cintra142": 0.536121673003802,
  "Test:base_Cintra143": 0.539923954372624,
  "Test:base_Cintra144": 0.5437262357414447,
  "Test:base_Cintra145": 0.5475285171102662,
  "Test:base_Cintra146": 0.5513307984790876,
  "Test:base_Cintra148": 0.5589353612167296,
  "Test:base_Cintra149": 0.5627376425855521,
  "Test:base_Cintra150": 0.5665399239543727,
  "Test:base_Cintra151": 0.5703422053231935,
  "Test:base_Cintra152": 0.5741444866920152,
  "Test:base_Cintra153": 0.5779467680608357,
  "Test:base_Cintra154": 0.5817490494296573,
  "Test:base_Cintra155": 0.5855513307984791,
  "Test:base_Cintra156": 0.5893536121673008,
  "Test:base_Cintra157": 0.5931558935361212,
  "Test:base_Cintra158": 0.5969581749049425,
  "Test:base_Cintra159": 0.6007604562737643,
  "Test:base_Cintra160": 0.6045627376425854,
  "Test:base_Cintra161": 0.6083650190114078,
  "Test:base_Cintra162": 0.6121673003802284,
  "Test:base_Cintra163": 0.615969581749049,
  "Test:base_Cintra164": 0.6197718631178707,
  "Test:base_Cintra165": 0.6235741444866912,
  "Test:base_Cintra166": 0.6273764258555133,
  "Test:base_Cintra167": 0.6311787072243358,
  "Test:base_Cintra168": 0.6349809885931562,
  "Test:base_Cintra169": 0.6387832699619767,
  "Test:base_Cintra170": 0.642585551330798,
  "Test:base_Cintra171": 0.6463878326996174,
  "Test:base_Cintra172": 0.6501901140684417,
  "Test:base_Cintra173": 0.653992395437262,
  "Test:base_Cintra174": 0.6577946768060839,
  "Test:base_Cintra175": 0.6615969581749045,
  "Test:base_Cintra176": 0.6653992395437258,
  "Test:base_Cintra177": 0.6692015209125464,
  "Test:base_Cintra178": 0.6730038022813708,
  "Test:base_Cintra179": 0.6768060836501868,
  "Test:base_Cintra180": 0.6806083650190117,
  "Test:base_Cintra181": 0.6844106463878322,
  "Test:base_Cintra182": 0.6882129277566531,
  "Test:base_Cintra183": 0.6920152091254733,
  "Test:base_Cintra184": 0.6958174904942973,
  "Test:base_Cintra185": 0.6996197718631177,
  "Test:base_Cintra186": 0.7034220532319374,
  "Test:base_Cintra187": 0.7072243346007637,
  "Test:base_Cintra188": 0.7110266159695826,
  "Test:base_Cintra189": 0.7148288973384025,
  "Test:base_Cintra190": 0.718631178707225,
  "Test:base_Cintra191": 0.7224334600760467,
  "Test:base_Cintra192": 0.7262357414448671,
  "Test:base_Cintra193": 0.7300380228136875,
  "Test:base_Cintra194": 0.7338403041825081,
  "Test:base_Cintra195": 0.7376425855513306,
  "Test:base_Cintra196": 0.7414448669201497,
  "Test:base_Cintra197": 0.7452471482889734,
  "Test:base_Cintra198": 0.7490494296577949,
  "Test:base_Cintra199": 0.7528517110266153,
  "Test:base_Cintra200": 0.7566539923954361,
  "Test:base_Cintra201": 0.7604562737642592,
  "Test:base_Cintra202": 0.764258555133078,
  "Test:base_Cintra203": 0.7680608365019019,
  "Test:base_Cintra204": 0.7718631178707158,
  "Test:base_Cintra205": 0.7756653992395429,
  "Test:base_Cintra206": 0.7794676806083684,
  "Test:base_Cintra207": 0.7832699619771852,
  "Test:base_Cintra208": 0.7870722433460138,
  "Test:base_Cintra209": 0.790874524714831,
  "Test:base_Cintra210": 0.794676806083648,
  "Test:base_Cintra211": 0.7984790874524759,
  "Test:base_Cintra212": 0.8022813688212963,
  "Test:base_Cintra213": 0.8060836501901127,
  "Test:base_Cintra214": 0.8098859315589376,
  "Test:base_Cintra215": 0.8136882129277554,
  "Test:base_Cintra216": 0.817490494296578,
  "Test:base_Cintra217": 0.8212927756654119,
  "Test:base_Cintra218": 0.8250950570342191,
  "Test:base_Cintra219": 0.8288973384030506,
  "Test:base_Cintra220": 0.8326996197718595,
  "Test:base_Cintra221": 0.8365019011406819,
  "Test:base_Cintra222": 0.8403041825095002,
  "Test:base_Cintra223": 0.8441064638783322,
  "Test:base_Cintra224": 0.8479087452471428,
  "Test:base_Cintra225": 0.8517110266159725,
  "Test:base_Cintra226": 0.8555133079847834,
  "Test:base_Cintra227": 0.8593155893536092,
  "Test:base_Cintra228": 0.8631178707224335,
  "Test:base_Cintra229": 0.8669201520912636,
  "Test:base_Cintra230": 0.8707224334600675,
  "Test:base_Cintra231": 0.8745247148289041,
  "Test:base_Cintra232": 0.8783269961977118,
  "Test:base_Cintra233": 0.8821292775665415,
  "Test:base_Cintra234": 0.8859315589353532,
  "Test:base_Cintra235": 0.8897338403041877,
  "Test:base_Cintra236": 0.8935361216729947,
  "Test:base_Cintra237": 0.8973384030418267,
  "Test:base_Cintra238": 0.901140684410634,
  "Test:base_Cintra239": 0.9049429657794688,
  "Test:base_Cintra240": 0.908745247148278,
  "Test:base_Cintra241": 0.9125475285171164,
  "Test:base_Cintra242": 0.9163498098859406,
  "Test:base_Cintra243": 0.9201520912547496,
  "Test:base_Cintra244": 0.923954372623586,
  "Test:base_Cintra245": 0.9277566539923904,
  "Test:base_Cintra246": 0.9315589353612244,
  "Test:base_Cintra247": 0.9353612167300316,
  "Test:base_Cintra248": 0.9391634980988668,
  "Test:base_Cintra249": 0.9429657794676788,
  "Test:base_Cintra25": 0.09125475285171104,
  "Test:base_Cintra250": 0.9467680608365008,
  "Test:base_Cintra251": 0.950570342205312,
  "Test:base_Cintra252": 0.9543726235741544,
  "Test:base_Cintra253": 0.9581749049429704,
  "Test:base_Cintra254": 0.961977186311795,
  "Test:base_Cintra255": 0.9657794676806072,
  "Test:base_Cintra256": 0.9695817490494358,
  "Test:base_Cintra257": 0.9733840304182504,
  "Test:base_Cintra258": 0.9771863117870776,
  "Test:base_Cintra259": 0.9809885931558822,
  "Test:base_Cintra260": 0.984790874524719,
  "Test:base_Cintra261": 0.9885931558935528,
  "Test:base_Cintra262": 0.9923954372623228,
  "Test:base_Cintra263": 0.9961977186311592,
  "Test:base_Cintra264": 1,
  "Test:base_Cintra27": 0.0988593155893536,
  "Test:base_Cintra31": 0.1140684410646388,
  "Test:base_Cintra34": 0.12547528517110265,
  "Test:base_Cintra38": 0.14068441064638784,
  "Test:base_Cintra39": 0.1444866920152091,
  "Test:base_Cintra40": 0.1482889733840304,
  "Test:base_Cintra41": 0.1520912547528517,
  "Test:base_Cintra44": 0.1634980988593156,
  "Test:base_Cintra47": 0.17490494296577946,
  "Test:base_Cintra48": 0.17870722433460076,
  "Test:base_Cintra51": 0.19011406844106463,
  "Test:base_Cintra55": 0.20532319391634984,
  "Test:base_Cintra57": 0.2129277566539924,
  "Test:base_Cintra58": 0.21673003802281368,
  "Test:base_Cintra60": 0.22433460076045628,
  "Test:base_Cintra63": 0.23574144486692017,
  "Test:base_Cintra65": 0.2433460076045627,
  "Test:base_Cintra67": 0.2509505703422053,
  "Test:base_Cintra70": 0.2623574144486692,
  "Test:base_Cintra71": 0.2661596958174905,
  "Test:base_Cintra75": 0.2813688212927757,
  "Test:base_Cintra78": 0.29277566539923955,
  "Test:base_Cintra79": 0.2965779467680608,
  "Test:base_Cintra80": 0.3003802281368821,
  "Test:base_Cintra81": 0.3041825095057034,
  "Test:base_Cintra82": 0.30798479087452474,
  "Test:base_Cintra83": 0.311787072243346,
  "Test:base_Cintra84": 0.3155893536121673,
  "Test:base_Cintra85": 0.3193916349809886,
  "Test:base_Cintra89": 0.33460076045627374,
  "Test:base_Cintra91": 0.34220532319391633,
  "Test:base_Cintra93": 0.349809885931559,
  "Test:base_Cintra94": 0.3536121673003801,
  "Test:base_Cintra95": 0.3574144486692015,
  "Test:base_Cintra97": 0.3650190114068441,
  "Test:base_Cintra98": 0.3688212927756655,
  "Test:base_Cintra99": 0.3726235741444867,
  "_runtime": 1712.4406335353851,
  "_step": 0,
  "_timestamp": 1737547641.0996037,
  "_wandb.runtime": 1716,
  "epoch": 0,
  "test_loss": -20.808238983154297,
  "test_loss_desurv": -17.28572654724121,
  "test_loss_values": -56.03337860107422,
  "trainer/global_step": 0
}

# Next event concordance

In [6]:
Cinter_keys = [_key for _key in raw_data_from_wandb.keys() if "Test:Cintra" in _key ]

decoded_cintra_diagnoses = {}
decoded_cintra_other = {}

for _key in Cinter_keys:
    _event = int(_key[len("Test:Cintra"):])                 # token
    _event_name = dm.decode([_event]).split(" ")[0]         # string
    _event_cintra = raw_data_from_wandb[_key]               # concordance

    if _event_name.upper() == _event_name:
        decoded_cintra_diagnoses = {**decoded_cintra_diagnoses, _event_name: _event_cintra}
    else:
        decoded_cintra_other = {**decoded_cintra_other, _event_name: _event_cintra}


In [7]:
# display(decoded_cintra_diagnoses)
# display(decoded_cintra_other)

In [8]:
BaseCinter_keys = [_key for _key in raw_data_from_wandb.keys() if "Test:base_Cintra" in _key ]

base_decoded_cintra_diagnoses = {}
base_decoded_cintra_other = {}
base_prevalence_diagnoses = {}
base_prevalence_other = {}

for _key in BaseCinter_keys:
    _event = int(_key[len("Test:base_Cintra"):])                 # token
    _event_name = dm.decode([_event]).split(" ")[0]         # string
    _event_cintra = raw_data_from_wandb[_key]               # concordance

    prevalence = dm.tokenizer._event_counts
    prevalence = prevalence.filter(pl.col("EVENT") ==_event_name)["COUNT"][0]

    if _event_name.upper() == _event_name:
        base_decoded_cintra_diagnoses = {**base_decoded_cintra_diagnoses, _event_name: _event_cintra}
        base_prevalence_diagnoses = {**base_prevalence_diagnoses, _event_name: prevalence}
    else:
        base_decoded_cintra_other = {**base_decoded_cintra_other, _event_name: _event_cintra}
        base_prevalence_other = {**base_prevalence_other, _event_name: prevalence}



In [9]:
# display(base_decoded_cintra_diagnoses)
# display(base_decoded_cintra_other)

In [10]:
keys_included_diagnoses = list(set(base_decoded_cintra_diagnoses.keys()) & set(decoded_cintra_diagnoses.keys()))
keys_included_other = list(set(base_decoded_cintra_other.keys()) & set(decoded_cintra_other.keys()))

In [11]:
for dict_name, result_dict, result_dict_base, result_dict_prev, keys_to_include in zip(["diagnoses", "other"],
                                                                     [decoded_cintra_diagnoses, decoded_cintra_other], 
                                                                     [base_decoded_cintra_diagnoses, base_decoded_cintra_other], 
                                                                     [base_prevalence_diagnoses, base_prevalence_other],
                                                                     [keys_included_diagnoses, keys_included_other]
                                                                     ):
    # plt.figure(figsize=(len(keys_to_include)/5,5))
    fig, ax1 = plt.subplots(figsize=(len(keys_to_include)/4,8))
    ax2 = ax1.twinx()  

    X_axis = np.arange(len(keys_to_include)) 

    Y_base = [result_dict_base[_key] for _key in keys_to_include]
    Y_survivEHR = [result_dict[_key] for _key in keys_to_include]
    Y_log_prevalence = [np.log(result_dict_prev[_key]) for _key in keys_to_include]
    # print(Y_log_prevalence)
    # print(np.min(Y_log_prevalence))
    # print(np.max(Y_log_prevalence))

    # Sort by prevalence
    arg_sort = np.argsort(Y_log_prevalence)
    Y_base = [Y_base[_i] for _i in arg_sort]
    Y_survivEHR = [Y_survivEHR[_i] for _i in arg_sort]
    Y_log_prevalence = [Y_log_prevalence[_i] for _i in arg_sort]
    keys_to_include = [keys_to_include[_i] for _i in arg_sort]

    width = 0.25
    ax1.bar(X_axis - width, Y_base, width, label = 'Concordance by prevalence', color="mediumblue") 
    ax1.bar(X_axis, Y_survivEHR, width, label = 'Concordance by SurvivEHR', color="firebrick") 
    ax2.bar(X_axis + width, Y_log_prevalence, width, label = 'Log-prevalence', color="darkseagreen") 

    ax1.set_xticks(X_axis, keys_to_include, rotation=90) 
    # ax1.xticks(X_axis, keys_to_include) 
    ax1.set_xlabel("Events") 
    ax1.set_ylabel("Self-supervised Concordance") 
    ax2.set_ylabel("Log Prevalence") 
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    ax1.set_ylim(0, 1.2)
    ax2.set_ylim(np.min(Y_log_prevalence)*0.95, np.max(Y_log_prevalence)*1.1)
    

    # plt.bar(result_dict.keys(), result_dict.values(), 0.5, color='g')
    # ax1.xticks()

    ybar = raw_data_from_wandb["Test:Cinter"]
    ax1.plot([0, len(result_dict)-1], 
             [ybar, ybar],
             label=f"SurvivEHR marginalised over events",
             color="firebrick")

    ybar = raw_data_from_wandb["Test:base_Cinter"]
    ax1.plot([0, len(result_dict)-1], 
             [ybar, ybar],
             label=f"Prevalence marginalised over events",
             color="mediumblue")
    
    plt.tight_layout()
    plt.savefig(f"{pre_trained_model}_inter_causal_eval_{dict_name}.png", bbox_inches="tight")
    plt.close()

# Future events

## SurvivEHR

In [12]:
Cinter_keys = [_key for _key in raw_data_from_wandb.keys() if "+" in _key and "base" not in _key ]
print(Cinter_keys)

x_survivEHR, y_survivEHR = [], []
for _key in Cinter_keys:
    x_survivEHR.append(int(_key[len("Test:Cinter+"):]) + 1 )                # steps ahead
    y_survivEHR.append(raw_data_from_wandb[_key] )                   # concordance


arg_sort = np.argsort(x_survivEHR)
x_survivEHR = [x_survivEHR[_i] for _i in arg_sort]
y_survivEHR = [y_survivEHR[_i] for _i in arg_sort]

print(x_survivEHR)
print(y_survivEHR)

['Test:Cinter+0', 'Test:Cinter+1', 'Test:Cinter+10', 'Test:Cinter+13', 'Test:Cinter+16', 'Test:Cinter+19', 'Test:Cinter+2', 'Test:Cinter+3', 'Test:Cinter+4', 'Test:Cinter+7']
[1, 2, 3, 4, 5, 8, 11, 14, 17, 20]
[0.953503257733534, 0.8958194011884513, 0.8744953913221967, 0.8559257247470172, 0.8257447465678294, 0.7530207013096748, 0.7646560061472428, 0.7330274976580149, 0.6945452986077362, 0.7070922089933495]


In [13]:
Cinter_keys = [_key for _key in raw_data_from_wandb.keys() if "+" in _key and "base" in _key ]
print(Cinter_keys)

x_base, y_base = [], []
for _key in Cinter_keys:
    x_base.append(int(_key[len("Test:base_Cinter+"):]) + 1 )                # steps ahead
    y_base.append(raw_data_from_wandb[_key] )                   # concordance

arg_sort = np.argsort(x_base)
x_base = [x_base[_i] for _i in arg_sort]
y_base = [y_base[_i] for _i in arg_sort]

['Test:base_Cinter+0', 'Test:base_Cinter+1', 'Test:base_Cinter+10', 'Test:base_Cinter+13', 'Test:base_Cinter+16', 'Test:base_Cinter+19', 'Test:base_Cinter+2', 'Test:base_Cinter+3', 'Test:base_Cinter+4', 'Test:base_Cinter+7']


In [14]:
plt.close()

plt.plot(x_survivEHR, y_survivEHR,
         label=f"SurvivEHR decay",
         color="firebrick")

plt.plot(x_base, y_base,
         label=f"Prevalence prognosis decay",
         color="mediumblue")

plt.legend()
plt.tight_layout()
plt.savefig(f"{pre_trained_model}_inter_decay.png", bbox_inches="tight")
    

## Comparison of decay across different pre-trained runs

Note: These are not fair comparisons as each run has been trained with different learning schedulers and for different lengths of time

In [15]:
all_survivEHR = {
    "SurvivEHR-cr-small-v1": (
        "baseline: 11M, 384 latent",
        [1, 2, 3, 4, 5, 8, 11, 14, 17, 20],
        [0.9845138238722124, 0.9366127214016844, 0.9158513102720236, 0.899843080451446, 0.868315386991299, 0.8239121250528096, 0.8252010439713312, 0.7866451755111037, 0.7641013179336177, 0.7604122318951223]
        ),
    "SurvivEHR-cr-small-192": (
        "11M, 192 latent",
        [1, 2, 3, 4, 5, 8, 11, 14, 17, 20], 
        [0.9530637980778408, 0.8965454649674218, 0.8732735785430125, 0.8556641921660932, 0.8248607427761983, 0.761096022692981, 0.7637418688146687, 0.7261393067724697, 0.7002773092426885, 0.7196441416973737]
        ),
    "SurvivEHR-cr-384-v1": (
        "129M, 384 latent",
        [1, 2, 3, 4, 5, 8, 11, 14, 17, 20],
        [0.9707854863671966, 0.9347402411296004, 0.9142678409102016, 0.8998330215060252, 0.8802228115580834, 0.8381073088297424, 0.856944131635776, 0.8165261475726017, 0.8077417879299013, 0.8017675470146958]
        ),
    "crPreTrain_small_1337": (
        "1M, 192 latent",
        [1, 2, 3, 4, 5, 8, 11, 14, 17, 20],
        [0.9785524581080296, 0.9322563387278594, 0.9193310330671408, 0.8942603657432556, 0.8602421957376115, 0.808292594604382, 0.8126680886580727, 0.7806386730589077, 0.7502644443809143, 0.7436763216230898]
        ),
                }


In [16]:
plt.close()

for run_id in all_survivEHR.keys():

    plt.plot(all_survivEHR[run_id][1], all_survivEHR[run_id][2],
             label=f"{all_survivEHR[run_id][0]}")

plt.plot(x_base, y_base,
         label=f"Prevalence prognosis decay",
         color="k")

plt.legend()
plt.title("Performance across pre-trained runs (note, different LR schedulers and run times)")
plt.tight_layout()
plt.savefig(f"all_inter_decay.png", bbox_inches="tight")
    

# Deprecated

In [ ]:
dm.tokenizer._event_counts

In [ ]:
save_path = f"/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/{cfg.experiment.run_id}/"

# Load Pre-Trained model

In [ ]:
ckpt_path = cfg.experiment.log_dir + f'checkpoints/{cfg.experiment.run_id}.ckpt'
model = SurvivalExperiment.load_from_checkpoint(ckpt_path)

# Initialise fine-tuning data module

In [ ]:
# Update dataset path to point to the new dataset 
cfg.data.path_to_ds = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"

# Build 
dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                            path_to_ds=cfg.data.path_to_ds,
                            load=True,
                            tokenizer="tabular",
                            batch_size=cfg.data.batch_size,
                            max_seq_length=cfg.transformer.block_size,
                            freq_threshold=cfg.data.unk_freq_threshold,
                            min_workers=cfg.data.min_workers,
                            overwrite_meta_information=cfg.data.meta_information_path,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size
print(f"{vocab_size} vocab elements")

# list of univariate measurements to model with Normal distribution
# Extract the measurements, using the fact that the diagnoses are all up upper case.
measurements_for_univariate_regression = [record for record in dm.tokenizer._event_counts["EVENT"] if record.upper() != record]
cfg.head.tokens_for_univariate_regression = dm.encode(measurements_for_univariate_regression) 
# display(measurements_for_univariate_regression)

In [ ]:
num_diagnoses = len(dm.train_set.meta_information["diagnosis_table"]["count"])
num_diagnosis_events = sum(dm.train_set.meta_information["diagnosis_table"]["count"])

is_medication = dm.train_set.meta_information["measurement_tables"]["count_obs"] == 0

num_medications = sum(is_medication)
num_medication_events = sum(dm.train_set.meta_information["measurement_tables"][is_medication]["count"])
num_measurement_test = sum(~is_medication)
num_measurement_test_events = sum(dm.train_set.meta_information["measurement_tables"][~is_medication]["count"])

num_measurement_test_events = sum(dm.train_set.meta_information["measurement_tables"][~is_medication]["count_obs"])

print(f'{num_diagnosis_events:,} diagnoses of {num_diagnoses} types')
print(f'{num_medication_events:,} medications of {num_medications} types')
print(f'{num_measurement_test_events:,} measurements and tests of {num_measurement_test} types')
print(f'{num_diagnoses+num_medication_events+num_measurement_test_events:,}')

print(f'{num_measurement_test_events:,}')
dm.train_set.meta_information.keys()

print(dm.train_set.tokenizer._event_counts)

In [ ]:
# # import pickle as pkl
# # import pathlib

# pkl_file_to_amend = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_test.pickle"

# with open(pkl_file_to_amend, 'rb') as pickle_file:
#     content = pickle.load(pickle_file)
# display(content)

# # new_dictionary = {}
# # for key in content.keys():
# #     str_to_remove = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/split=val/"
# #     new_key = str(key)[len(str_to_remove):]
# #     new_dictionary[new_key] = content[key]
# # display(new_dictionary)


# # with open(pkl_file_to_amend, 'wb') as handle:
# #     pickle.dump(new_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# new_dictionary

In [ ]:
import copy
start = time.time()   # starting time
for batch in dm.train_dataloader():
    # print(batch["tokens"][1,:])
    
    c_batch = convert_batch_to_none_causal(batch)
    # print(c_batch["tokens"][1,:])
    # print(c_batch["target_token"][1])

    # print(batch["tokens"][1,:])
    
    break
    
print(f"batch loaded in {time.time()-start} seconds")    
    
# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

In [ ]:
display(batch.keys())
display(c_batch.keys())

print(batch["static_covariates"].shape)

# print(dm.train_set.static_1hot)
# print(dm.train_set.static_1hot["SEX"].categories_)
# print(dm.train_set.static_1hot["IMD"].categories_)
# print(dm.train_set.static_1hot["ETHNICITY"].categories_)

print(batch["tokens"][1,:])
print(c_batch["tokens"][1,:])
print(c_batch["target_token"][1])

## View an example sample

In [ ]:
dm.test_set.view_sample(11003, max_dynamic_events=None, report_time=True)

# Custom wrapper prediction last token

To begin with, I will just loop over samples individually to test the zero-shot capacity of SurvivEHR. 

In [ ]:


# Verifying on datamodule 
for _idx, batch in enumerate(dm.test_dataloader()):
    if _idx > 10:
        break
    print(_idx)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))
    batch = replace_last_non_pad_with_pad(batch)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))

In [ ]:
outcome_of_interest = ["COPD", "SUBSTANCEMISUSE"]
outcome_token = dm.encode(outcome_of_interest)[0]
print(outcome_token)
# print(model(batch))

In [ ]:
Hs, labels = [], []
mins,maxes=[],[]
for _idx, batch in enumerate(dm.test_dataloader()):

    batch = replace_last_non_pad_with_pad(batch)
    print(batch["tokens".shape)
    outputs, _, hidden_states = model(batch, is_generation=True)
    print(outputs)
    
    hidden_states = hidden_states.cpu().detach().numpy()                           # (64, 128, 384) 
    Hs.append( hidden_states.reshape(hidden_states.shape[0], -1) )
    labels.append((batch["target_token"] == outcome_token).long().numpy())

    if _idx == 9:
        break



# Visualise hidden dimension labelled by target

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

H = np.concatenate(Hs, 0)
lbl = np.concatenate(labels, 0)

H = StandardScaler().fit_transform(H)
reducer = umap.UMAP()
H_proj = reducer.fit_transform(H)

plt.close()
plt.scatter(H_proj[:,0], H_proj[:,1], c=lbl)
plt.savefig(save_path + f"zero_shot/hidden_umap.png")

In [ ]:
print(outputs["surv"]["surv_CDF"][outcome_token].shape)

# The first two tokens in the vocab correspond to the PAD and UNK tokens. There is no CDF corresponding to the PAD token, so the indexing for surv_CDF begins as ["UNK", "ADDISONS_DISEASE", ...]
# print(dm.decode([0,1,2]))

outcomes = ["COPD", "SUBSTANCEMISUSE"]
outcome_tokens = dm.encode(outcomes)

# for outcome in outcomes:
    # observed_outcome_token = dm.encode([outcome])[0]
cdf = np.zeros_like(outputs["surv"]["surv_CDF"][0])
lbls = np.zeros_like(batch["target_token"])

for _outcome_token in outcome_tokens:
    cdf += outputs["surv"]["surv_CDF"][_outcome_token - 1] 
    lbls += (batch["target_token"] == _outcome_token).long().numpy()

plt.close()
cdf_true = cdf[lbls==1,:]
cdf_false = cdf[lbls==0,:]
for i in range(cdf_true.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_true[i,:], c="r", label="outcome occurred next" if i == 0 else None, alpha=1)
for i in range(cdf_false.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_false[i,:], c="k", label="outcome did not occur next" if i == 0 else None, alpha=0.3)

plt.legend(loc=2)
plt.xlabel("days")
plt.ylabel(f"P(t>T) - outcomes={','.join(outcomes)}")
plt.savefig(save_path + f"zero_shot/cdf_outcomes.png")

In [ ]:
print(batch["target_token"].unique())
print(len(outputs["surv"]["surv_CDF"]))

In [ ]:
dm.decode([2])

In [ ]:
outputs["surv"]["surv_CDF"][observed_outcome_token - 1]